In [1]:
import pandas as pd
import numpy as np
import git
from datetime import datetime as dt
from connect_to_mysql import Database
import networkx as nx
import matplotlib.pyplot as plt
import collections 
pd.set_option("display.max_columns", None)

In [ ]:
#repo = git.Repo(".", search_parent_directories=True).git.rev_parse("--show-toplevel")
#df = pd.read_csv(f"{repo}/data/cleaning/test_1651834681.csv")
df = pd.read_csv("/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/mtd_combined.csv")
df = df[(df["state"] == "completed")]

In [ ]:
Database.query_dataframe('rides', 'Show tables')

In [ ]:
#df_edges = Database.query_dataframe('stopconnection', "Select * From stopconnection")
df_edges = pd.read_excel("/Users/ericchittka/Documents/Daten/MoDstops+Preismodell.xlsx")

In [ ]:
df_edges.dtypes

### Number of Drives 

In [ ]:
df.columns

In [ ]:
pd.to_datetime(df["scheduled_to"])[0].month

In [ ]:
frequencies = pd.DataFrame(df[(pd.to_datetime(df["scheduled_to"]).dt.month == 2) & (pd.to_datetime(df["scheduled_to"]).dt.year == 2022) ].groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()

frequencies

In [ ]:
frequencies["pickup_address"] = frequencies["pickup_address"].astype(str)
frequencies["dropoff_address"] = frequencies["dropoff_address"].astype(str)

In [ ]:
#df_edges = pd.read_excel(
#        f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name="Liste 2022"
#)

In [ ]:
df_edges.columns

In [ ]:
df_edges = df_edges[["id_start", "id_end"]]

In [ ]:
df_edges["id_start"] = df_edges["id_start"].astype(str)
df_edges["id_end"] = df_edges["id_end"].astype(str)

In [ ]:
edges = pd.merge(df_edges, frequencies,  how='left', left_on=['id_start','id_end'], right_on = ['pickup_address','dropoff_address'])[["id_start", "id_end", "number_of_drives"]]
edges = edges[edges["number_of_drives"].isna() == False]
edges["number_of_drives"] = 28  / edges["number_of_drives"] 

In [ ]:
edges

In [ ]:
edges["number_of_drives"].fillna(0, inplace=True)

In [ ]:
#G = nx.from_pandas_edgelist(
#    edges,
#    source = "id_start",
#    target = "id_end",
#    edge_attr= "number_of_drives"
#)

In [ ]:
#plt.figure(3,figsize=(30,30)) 
#nx.draw(G)
#plt.show()

In [ ]:
#print(nx.shortest_path(G, source="10002", target="4007", weight = "number_of_drives"))
#print(nx.shortest_path_length(G, source="10002", target="4007", weight="number_of_drives", method='dijkstra'))

### Average Ride Time per drive  

In [ ]:
#df
#df.dtypes['ride_time']

In [ ]:
average = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean())
average.rename(columns = {"ride_time":"avg_ride_time"}, inplace = True)
average

In [ ]:
average["pickup_address"] = average["pickup_address"].astype(str)
average["dropoff_address"] = average["dropoff_address"].astype(str)

In [ ]:
edges2 = pd.merge(df_edges, average,  how='left', left_on=['id_start','id_end'], right_on = ['pickup_address','dropoff_address'])[["id_start", "id_end", "avg_ride_time"]]
edges2 = edges2[edges2["avg_ride_time"].isna() == False]
edges2

In [ ]:
#G1 = nx.from_pandas_edgelist(
#    edges2,
#    source = "id_start",
#    target = "id_end",
#    edge_attr= "avg_ride_time"
#)

In [ ]:
#plt.figure(3,figsize=(30,30)) 
#nx.draw(G1)
#plt.show()

In [ ]:
#print(nx.shortest_path(G1, source="1001", target="9015", weight = "avg_ride_time"))
#print(nx.shortest_path_length(G1, source="1001", target="9015", weight="avg_ride_time", method='dijkstra'))

### Original Function

In [ ]:
def get_shortest_ride(startpoint, endpoint, start_date, end_date):
    start_date = dt.strptime(start_date, "%m/%d/%Y")
    end_date = dt.strptime(end_date, "%m/%d/%Y")
    days = (end_date - start_date).days + 1
    df = Database.query_dataframe('rides', "Select * From rides where state = 'completed'")
    df = df[(df["scheduled_to"] > start_date) & (df["scheduled_to"] < end_date)]
    
    drives = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()
    
    drives["waiting_time"] = days / drives["number_of_drives"]

    drives["avg_ride_time"] = df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean()["ride_time"] / 60 / 60 / 24
    
    drives["avg_time_to_destination"] = drives["waiting_time"] + drives["avg_ride_time"]

    G = nx.from_pandas_edgelist(
        drives,
        source = "pickup_address",
        target = "dropoff_address",
        edge_attr= "avg_time_to_destination"
    )

    path = nx.shortest_path(G, source=startpoint, target=endpoint, weight = "avg_time_to_destination")
    shortest_time = nx.shortest_path_length(G, source=startpoint, target=endpoint, weight="avg_time_to_destination", method='dijkstra') 

    return (path, shortest_time)

### Set start and end date 

In [2]:
df = pd.read_csv("/Users/ericchittka/Documents/GitHub/vehicle-stream-team-project/data/mtd_combined.csv")
df = df[(df["state"] == "completed")]
pd.to_datetime(df["scheduled_to"])[0].month
df

,Unnamed: 0,id,user_id,distance,number_of_passenger,price_operations,price_offer,price_payed,free_ride,payment_type,pickup_address,dropoff_address,state,created_from_offer,created_at,scheduled_to,dispatched_at,pickup_arrival_time,arriving_push,vehicle_arrived_at,earliest_pickup_expectation,pickup_first_eta,pickup_eta,pickup_at,dropoff_first_eta,dropoff_eta,dropoff_at,updated_at,arrival_deviation,waiting_time,boarding_time,ride_time,trip_time,shortest_ridetime,delay,longer_route_factor,arrival_indicator,rating,rating_puenktlichkeit,rating_sauberkeit,rating_fahrer,rating_find_modstop,rating_other_comments,cancellation_reason,cancellation_comment,bahn_card_number,year_card_type,year_card_number,canceled_at,rating_question_one,rating_question_two
0,0,5727475e-8224-4302-9228-c92b9d4a5220,f8ff0526-887a-4e48-ad96-977e12fd70c1,5483,1.0,4.65,4.65,0.00,0.0,STANDARD,Würzmühle,Globus,completed,NaN,2021-06-30 23:12:47,2021-07-01 07:30:00,2021-07-01 07:22:00,00:10:37,2021-07-01 07:29:37,2021-07-01 07:32:37,2021-07-01 07:25:00,2021-07-01 07:33:55,2021-07-01 07:34:07,2021-07-01 07:33:08,2021-07-01 07:44:27,2021-07-01 07:44:39,2021-07-01 07:44:44,2021-11-03 16:00:44,0.0,00:07:37,00:00:31,00:11:36,00:19:13,00:10:57.960000,00:08:15,1.06,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,bb916271-0627-4196-8ec1-5324e0e1f71d,f07028da-ca7e-4713-9e45-743c71712e80,3040,1.0,3.45,1.55,1.55,0.0,VRN,Jüdischer Friedhof,Globus,completed,NaN,2021-07-01 09:21:39,NaN,2021-07-01 09:21:39,00:12:59,2021-07-01 09:31:38,2021-07-01 09:34:38,2021-07-01 09:24:39,2021-07-01 09:32:07,2021-07-01 09:36:59,2021-07-01 09:35:40,2021-07-01 09:40:59,2021-07-01 09:45:53,2021-07-01 09:42:27,2021-11-15 11:24:38,0.0,00:09:59,00:01:02,00:06:47,00:16:46,00:06:04.800000,00:10:41,1.12,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,f797b4d6-2fd4-442a-aec3-32c7f34c9b3e,7b093b19-1bc2-4bc0-b9cb-fcb8cfb8f074,6511,1.0,5.20,3.10,3.10,0.0,VRN,Schütt,Kanzelkopfstraße,completed,NaN,2021-07-01 17:11:11,NaN,2021-07-01 17:11:11,00:10:45,2021-07-01 17:18:56,2021-07-01 17:21:56,2021-07-01 17:14:11,2021-07-01 17:24:39,2021-07-01 17:23:01,2021-07-01 17:22:06,2021-07-01 17:36:25,2021-07-01 17:34:47,2021-07-01 17:32:41,2021-11-15 11:24:38,0.0,00:07:45,00:00:10,00:10:35,00:18:20,00:13:01.320000,00:05:19,0.81,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,5b9be6cf-1b25-43d7-b077-e0ede60ba49a,f07028da-ca7e-4713-9e45-743c71712e80,3040,1.0,3.45,1.55,1.55,0.0,VRN,Globus,Jüdischer Friedhof,completed,NaN,2021-07-01 18:24:13,NaN,2021-07-01 18:24:13,00:08:09,2021-07-01 18:29:22,2021-07-01 18:32:22,2021-07-01 18:27:13,2021-07-01 18:30:25,2021-07-01 18:34:21,2021-07-01 18:33:23,2021-07-01 18:38:27,2021-07-01 18:42:23,2021-07-01 18:39:50,2021-11-15 11:24:38,0.0,00:05:09,00:01:01,00:06:27,00:11:36,00:06:04.800000,00:05:31,1.06,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,90ef7121-ff20-40bb-9489-f9e935519443,7b093b19-1bc2-4bc0-b9cb-fcb8cfb8f074,5974,1.0,4.90,2.80,2.80,0.0,VRN,Kanzelkopfstraße,Tanzmanufaktur,completed,NaN,2021-07-01 18:35:04,NaN,2021-07-01 18:35:04,00:18:55,2021-07-01 18:51:18,2021-07-01 18:53:59,2021-07-01 18:38:04,2021-07-01 18:53:37,2021-07-01 18:55:14,2021-07-01 18:53:59,2021-07-01 19:04:18,2021-07-01 19:05:55,2021-07-01 19:04:05,2021-11-15 11:24:38,-19.0,00:15:55,00:00:00,00:10:06,00:26:01,00:11:56.880000,00:14:04,0.85,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12564,12564,04abf588-66fd-4891-b3e0-742b54ae850f,76f7bb28-ae72-4d6b-83f5-ce252a641f81,2983,1.0,3.40,3.40,3.40,0.0,STANDARD,Alter Turnplatz,Zur Kapelle,completed,4227.0,2021-08-21 19:05:17,NaN,NaN,NaN,2021-08-21 19:12:16,NaN,NaN,2021-08-21 19:16:23,2021-08-21 19:17:14,2021-08-21 19:14:55,2021-08-21 19:24:28,2021-08-21 19:25:21,2021-08-21 19:20:25,202

In [3]:
#df = Database.query_dataframe('rides', "Select * From rides where state = 'completed'")
#startdate = np.min([np.nonzero(df.scheduled_to)])
startdate = df['scheduled_to'].dropna().min()
startdate = dt.strptime(startdate, '%Y-%m-%d %H:%M:%S')
#start_date = startdate.strftime("%m/%d/%Y")
start_date = startdate.strftime('%m/%d/%Y')
#enddate = np.amax(df.scheduled_to.values)
enddate = df['scheduled_to'].dropna().max()
enddate = dt.strptime(enddate, '%Y-%m-%d %H:%M:%S')
end_date = enddate.strftime('%m/%d/%Y')
#enddate = enddate.strftime('%m/%d/%Y')
#end_date = enddate.strftime("%m/%d/%Y")

In [ ]:
#df['pickup_address'] = df['pickup_address'].map(str)
#df['dropoff_address'] = df['dropoff_address'].map(str)

### Drive DF

In [4]:
end_date = dt.strptime(end_date, '%m/%d/%Y')
start_date = dt.strptime(start_date, '%m/%d/%Y')

In [10]:
days = (end_date - start_date).days + 1
drives = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()
drives["waiting_time"] = days / drives["number_of_drives"]
drives["avg_ride_time"] = df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean()["ride_time"] / 60 / 60 / 24
drives["avg_time_to_destination"] = drives["waiting_time"] + drives["avg_ride_time"]

/var/folders/kw/t27bnrpn52lfl7x3g64tdw1r0000gn/T/ipykernel_52450/1705694110.py:4: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  drives["avg_ride_time"] = df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean()["ride_time"] / 60 / 60 / 24


KeyError: 'ride_time'

In [11]:
drives.dtypes

pickup_address       object
dropoff_address      object
number_of_drives      int64
waiting_time        float64
dtype: object

### Graph

In [ ]:
G = nx.from_pandas_edgelist(
    drives,
    source = "pickup_address",
    target = "dropoff_address",
    edge_attr= "avg_time_to_destination"
)

In [ ]:
G.nodes

In [ ]:
def calculate_graph(start_date, end_date):
  
    G = nx.from_pandas_edgelist(
        drives,
        source = "pickup_address",
        target = "dropoff_address",
        edge_attr= "avg_time_to_destination"
    )
    return G

In [ ]:
def get_shortest_ride(startpoint, endpoint, G):

    if startpoint not in G:
        return 'Not in G'
    elif endpoint not in G: 
        return 'Not in G'
    else:
        path = nx.shortest_path(G, source=startpoint, target=endpoint, weight = "avg_time_to_destination")
        return path

In [ ]:
df_test = pd.DataFrame({'Start':['1001', '1002', '1931391'], 'End': ['10004', '1008', '32432432']})
df_test["Start"] = df_test["Start"].astype(str).astype(int)
df_test["End"] = df_test["End"].astype(str).astype(int)

In [ ]:
df_test['HotSpots'] = df_test.apply(lambda x: get_shortest_ride(x.Start, x.End, G), axis=1)

In [ ]:
df_edges['HotSpots'] = df_edges.apply(lambda x: get_shortest_ride(x.id_start, x.id_end, G), axis=1)

In [ ]:
df_edges

In [ ]:
hotSpots_list = list(df_edges['HotSpots'])

In [ ]:
flat_hotSpots_list = [item for sublist in hotSpots_list for item in sublist]

In [ ]:
flat_hotSpots_list

In [ ]:
counter=collections.Counter(flat_hotSpots_list)
print(counter.most_common(20))